# Reto 01 - Prompt Engineering

Consejo rápido: Para ver la Tabla de Contenidos de este Notebook en VS Code, en la pestaña "Explorer", expande la sección "Outline".

## 1. Experimentación de Parámetros
Primero configuraremos el Desafío. Carga la clave API y las bibliotecas de Python relevantes utilizando las celdas a continuación.

### 1.1 Preparación
#### 1.1.1 Configuración del Entorno


#### Crea o actualiza el archivo de entorno llamado ".env"
Hemos incluido un archivo de entorno de ejemplo para tu referencia. Puedes encontrar el archivo `.env.sample` en la raiz de este repositorio o en la carpeta `Student/Resources` dentro de la documentación del desafío. Siéntete libre de hacer las modificaciones necesarias y renombra el archivo como `.env`.

**NOTA:** Asegúrate de mantener el nombre de cada modelo igual al nombre de implementación del modelo que configuraste en Azure OpenAI.

**NOTA:** Ten en cuenta que algunos modelos de AOAI podrían estar obsoletos. Deberías crear los que estén disponibles según sea necesario. Para completar este primer desafío, gpt-35-turbo es el único modelo que necesitas.

Para obtener información sobre los modelos actuales, consulta: [Modelos del Servicio Azure OpenAI Service](https://learn.microsoft.com/es-mx/azure/ai-services/openai/concepts/models)

####  Configuración del Entorno de Azure OpenAI

In [37]:
import openai
import os
import json
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [38]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

chat_model=os.getenv("CHAT_MODEL_NAME")


#### 1.1.2 Explicación de Parámetros
Especificamos los hiperparámetros para los modelos de Azure OpenAI dentro de las funciones de ayuda. Los usuarios pueden ajustar los parámetros según sus necesidades.


###### **Temperatura**
La temperatura varía de 0 a 2. Aquí tienes un desglose rápido de cómo funciona:
- Temperatura baja (0 a 0.3): Salidas más enfocadas, coherentes y conservadoras.
- Temperatura media (0.3 a 0.7): Creatividad y coherencia equilibradas.
- Alta temperatura (> 0.7): Altamente creativa y diversa, pero potencialmente menos coherente.

###### **Top_p**
Establece el límite de probabilidad para el muestreo de tokens, afectando la amplitud de opciones que considera la IA. Valores más altos conducen a más aleatoriedad, mientras que valores más bajos resultan en salidas más enfocadas.


El parámetro "top_p" actúa como un filtro que controla cuántas palabras o frases diferentes considera el modelo de lenguaje cuando intenta predecir la siguiente palabra. Si configuras el valor de "top p" a 0.5, el modelo de lenguaje solo considerará las 50 palabras o frases más probables que podrían seguir. Pero si configuras el valor de "top p" a 0.9, el modelo de lenguaje considerará las 90 palabras o frases más probables.

Dado que "top_p" y "temperatura" realizan trabajos similares como hiperparámetros, usualmente solo ajustamos uno de ellos en lugar de ambos al mismo tiempo.

###### **Max_tokens**
Max_tokens determina la longitud máxima del texto generado. Al establecer un límite, puedes controlar cuánto texto devolverá el modelo LLM, asegurando que no proporcione una respuesta demasiado larga.

###### **Frequency_penalty**
La penalización de frecuencia asegura que el texto generado sea variado al dar una penalización a los tokens que ya se han utilizado en la respuesta.

Varía de -2.0 a 2.0, con valores más altos resultando en una salida más diversa.

Los valores positivos penalizan nuevos tokens basados en su frecuencia existente en el texto hasta el momento, disminuyendo la probabilidad del modelo de repetir la misma línea textualmente.

#### 1.1.3 Función de Ayuda
A lo largo de este desafío, utilizaremos el modelo `gpt-3.5-turbo` de Azure OpenAI y el [punto final de conexión (endpoint) de chat completions](https://platform.openai.com/docs/guides/chat). 

Esta función de ayuda facilitará el uso de prompts y la visualización de las salidas generadas.

**get_chat_completion** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección.

**get_completion_from_messages** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección, habilitando el historial de chat.


In [39]:
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

In [40]:
def get_completion_from_messages(messages, model=chat_model, temperature=0.8):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]



#### Prueba las funciones auxiliares

In [41]:
messages =  f"""
tell me a joke.
"""
response = get_chat_completion(messages)
print(response)

Why couldn't the bicycle stand up by itself?

Because it was two tired!


In [42]:
messages =  [      
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'Why did the chicken cross the road'},   
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side!


#### Tareas para Estudiantes (preguntas abiertas):
1. Crea una función de finalización de texto (completions) para una temperatura muy alta, entre (0,1).
2. Crea una función de finalización de texto (completions) para un valor muy pequeño de max_tokens a fin de obtener respuestas más cortas.
3. Crea funciones de finalización (completions) con 2 valores diferentes de penalización por diversidad entre (0,2).

Prueba las funciones de finalización (completions) que has creado y compara los resultados obtenidos con las pruebas anteriores.

In [43]:
# Prueba aquí la función de completación para una temperatura muy alta entre (0,1)
def get_chat_completion(prompt, model=chat_model, temperature=0.5):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

# Test with different temperatures
temperatures = [0.0, 0.2, 0.5, 0.8, 1.0]
messages = f"""
Imagine you are a storyteller. Tell me a short story about a dragon and a knight. Use 50 words.
"""

for temp in temperatures:
    response = get_chat_completion(messages, temperature=temp)
    print(f"Response with temperature {temp}:\n{response}\n{'-'*50}\n")

Response with temperature 0.0:
Once upon a time, a fierce dragon terrorized the kingdom. A brave knight, Sir William, accepted the challenge to defeat the beast. After a fierce battle, the knight emerged victorious, earning the respect of the dragon. They became unlikely friends, protecting the kingdom together for years to come.
--------------------------------------------------

Response with temperature 0.2:
In a land of magic and mystery, a fearsome dragon terrorized the kingdom. A brave knight, armed with courage and determination, set out to defeat the beast. After a fierce battle, the knight emerged victorious, earning the respect of the people and bringing peace to the land once more.
--------------------------------------------------

Response with temperature 0.5:
Once upon a time, a fearsome dragon terrorized the kingdom. The brave knight, Sir Edward, accepted the challenge to defeat the beast. After a fierce battle, the knight emerged victorious, earning the admiration of t

In [44]:
# Prueba aquí la función de completación para un valor max_tokens muy bajo para obtener respuestas más cortas
def get_chat_completion(prompt, model=chat_model, temperature=0.5, max_tokens=100):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
        max_tokens = max_tokens,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

# Test with different temperatures
max_tokens = [10, 50, 100, 150, 200]
messages = f"""
Tell me a short story
"""

for token in max_tokens:
    response = get_chat_completion(messages, max_tokens=token)
    print(f"Response with max_tokens {token}:\n{response}\n{'-'*50}\n")

Response with max_tokens 10:
Once upon a time, in a small village nestled
--------------------------------------------------

Response with max_tokens 50:
Once upon a time, in a small village nestled in the mountains, there lived a young girl named Lily. She was known throughout the village for her kindness and her love for animals. Every day, she would spend hours in the meadows, playing
--------------------------------------------------

Response with max_tokens 100:
Once upon a time, in a small village nestled at the foot of a majestic mountain, there lived a young girl named Lily. Lily was known throughout the village for her kindness and generosity, always willing to lend a helping hand to those in need.

One day, a terrible storm swept through the village, causing widespread damage and leaving many families in need of assistance. Without hesitation, Lily sprang into action, gathering supplies and organizing a relief effort to help her fellow villagers rebuild their homes.

As the

In [45]:
# Prueba aquí las funciones de completación con 2 valores diferentes de penalización por diversidad entre (0,2).
def get_chat_completion(prompt, model=chat_model, temperature=0.5, max_tokens=200, p=1.0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
        max_tokens = max_tokens,
        top_p = p
    )
    return response.choices[0].message["content"]

# Test with different temperatures
top_p = [0.1, 0.3, 0.5, 0.7, 1.0]
messages = f"""
Tell me a short story
"""

for p in top_p:
    response = get_chat_completion(messages, p=p)
    print(f"Response with top_p {p}:\n{response}\n{'-'*50}\n")

Response with top_p 0.1:
Once upon a time, in a small village nestled in the mountains, there lived a young girl named Lily. She was known for her kindness and generosity, always willing to help those in need.

One day, a terrible storm hit the village, causing widespread damage and leaving many families homeless. Lily knew she had to do something to help. She gathered her friends and together they started a relief effort, collecting food, clothing, and supplies for those affected by the storm.

As they worked tirelessly to distribute the donations, word of their efforts spread throughout the village. Soon, more and more people joined in, offering their support and assistance. The village came together in a way it never had before, united in their mission to help their neighbors in need.

Thanks to Lily's leadership and the community's collective efforts, the village was able to rebuild and recover from the storm in record time. And though the storm had caused destruction, it also brou

### 1.2 Ingeniería de Mensajes del Sistema
Los usuarios pueden lograr que los modelos respondan en el tono deseado ajustando el mensaje del sistema.

#### 1.2.1 Cambio de Tono

In [46]:
messages =  [      
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages)
print(response)

Sure, here's a joke for you: Why did the scarecrow win an award? Because he was outstanding in his field!


In [47]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages)
print(response)

Why did the clown go to the doctor?

Because he was feeling a little funny!


#### Tarea para Estudiantes (pregunta abierta):
Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.


In [48]:
# Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks as programmer.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages)
print(response)

Why do programmers prefer dark mode? Because the light attracts bugs!


#### 1.2.2 Recordar el Nombre de la Empresa

In [49]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages)
print(response)

I'm sorry, I don't have that information. Could you please provide me with the name of the company you work for?


#### Tarea para Estudiantes:
Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.

In [50]:
# Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.
messages = [
    {'role': 'system', 'content': 'You are a helpful chatbot. You always remember and provide the name of the company the user works for, which is "Spider Inc."'},
    {'role': 'user', 'content': 'Hi, my name is Mel.'},
    {'role': 'assistant', 'content': "Hi! It's nice to meet you. Is there anything I can help you with today?"},
    {'role': 'user', 'content': 'Yes, can you remind me which company I work for?'}
]

response = get_completion_from_messages(messages)
print(response)

Of course! You work for Spider Inc. How can I assist you further, Mel?


## 2. Principios de Prompting Iterativo
Es una buena práctica realizar un prompting iterativo para que el modelo pueda generar la respuesta más adecuada según las especificaciones del usuario.
- **Principio 1: Escribe instrucciones claras y específicas**
- **Principio 2: Dale tiempo al modelo para "pensar"**



### 2.1 Escribe instrucciones claras y específicas

#### Táctica 1: Delimitadores

#### Tarea para Estudiantes:
Usa delimitadores para indicar claramente las partes distintas de la entrada.

Los delimitadores pueden ser cualquier cosa como: ````, """, < >, `<tag> </tag>`, `:`

In [51]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""




In [52]:
# Usa delimitadores para indicar claramente las partes distintas de la entrada y pide al modelo que resuma el texto.
text = f"""
### Start of Text ###
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013. Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion. The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
### End of Text ###

### Task: Summarize the above information ###
"""
message = [{'role': 'system', 'content': 'You are an assistant that summarizes text clearly.'},
           {'role': 'user', 'content': text}]

# Aquí llamas al modelo para resumir el texto
response = get_completion_from_messages(message)

print(response)

The 2020 Summer Olympics, held in 2021 in Tokyo, Japan, were unique due to the postponement caused by the COVID-19 pandemic. The event, the most expensive Olympics ever, took place without spectators. New sports and events were introduced, and the United States topped the medal count. Japan set a record for the most medals won by the host nation, and several countries won their first-ever Olympic gold medals.


#### Táctica 2: Resumen: especificar el número de palabras, extraer información

#### Texto a resumir

In [53]:
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

#### Tarea para Estudiantes:
Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.

In [54]:
# Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.
message = [{'role': 'system', 'content': 'You are an assistant that summarizes text clearly. Do what is inside the (``` ```) to personalize the summarize '},
           {'role': 'user', 'content': f'summarize the {game_review}, ``` limit to only 20 words ``` '}]

# Aquí llamas al modelo para resumir el texto
response = get_completion_from_messages(message)

print(response)

The 2020 Summer Olympics in Tokyo was postponed to 2021 due to COVID-19, featured new sports, and saw various countries excel.


#### Tarea para Estudiantes:
Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")

In [55]:
# Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")
message = [{'role': 'system', 'content': 'You are an assistant that extract inforrmation from text clearly. EXtract what is inside the (``` ```)'},
           {'role': 'user', 'content': f'Extract from {game_review}, ``` How many gold edals won the Great Britain Team ``` '}]

# Aquí llamas al modelo para resumir el texto
response = get_completion_from_messages(message)

print(response)

Great Britain Team won 22 gold medals.


#### Táctica 3: Inferencia: solicitar emociones, sentimientos o temas

#### Tarea para Estudiantes:
Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña.

Usa un objeto JSON.

In [56]:
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

In [57]:
# Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña. Usa un objeto JSON
message = [{'role': 'system', 'content': '''
You're an assistant designed to analyze sentiment from text data.
Users will paste in a string of text and you'll respond with an assessment of the response of the speaker.
Rate on a scale of 1-10 (10 being highest). Explain why this rating was given.
Use a JSON object to print.
'''},
{'role': 'user', 'content': f'Analyze the sentiments from {review}'}]

response = get_completion_from_messages(message)

print(response)

{
  "rating": 8,
  "reason": "The sentiments expressed by different individuals vary in their assessment of the Olympic ceremony, ranging from dignified and appropriate to an uneven mix of contrasts. The positive remarks about the ceremony being dignified and expressing gratitude are balanced by criticisms of it being low-key and lacking in jubilation. The mention of the updated Olympic motto and the importance of inclusion and diversity add depth to the analysis. Overall, the mixed reactions and insightful commentary warrant a high rating of 8."
}


#### Desafío para Estudiantes:
Deduce 3 temas (tópicos) de la historia a continuación.

In [58]:
story = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

In [59]:
# Deduce 3 temas (tópicos) de la historia a continuación.
message = [{'role': 'system', 'content': '''
You're an assistant designed to deduce the topic from text.
Users will paste in a string of text and you'll respond with topics from the text given.
Use the format: Topic: .
'''},
{'role': 'user', 'content': f'Deduce 3 topics from {story}'}]

# Aquí llamas al modelo para resumir el texto
response = get_completion_from_messages(message)

print(response)

Topic: 2020 Summer Olympics in Tokyo
Topic: Impact of COVID-19 on the Olympics
Topic: New sports and events introduced in the Olympics


#### Táctica 4: Transformación: especifica el idioma objetivo y el estilo de escritura, y solicita una revisión gramatical.

#### Tarea para Estudiantes:
Traductor Universal

Personas de todo el mundo desean conocer las noticias de los Juegos Olímpicos en su idioma nativo. En este caso, las noticias deben ser traducidas a diferentes idiomas. Traduce cada elemento de noticias a continuación tanto al coreano como al inglés.

In [60]:
news = [
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
"比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"]

In [61]:
# Escribe aquí el código para la tarea .
message = [{'role': 'system', 'content': '''
You're an assistant designed to translate the given text to the requested language.
Users will paste in a string of text and you'll translate to the requested language.
Use the format: <given language> -> <requested language>:\n <Original text and translated text>
'''},
{'role': 'user', 'content': f'Translate form english to korean and from korean to english {news}'}]

response = get_completion_from_messages(message)

print(response)

English -> Korean:
'팔레스타인은 2021년 6월 28일 FINA 포인트 시스템을 기반으로, 각각의 개인 이벤트에서 상위 랭킹 수영 선수 두 명(성별당 한 명)을 올림픽에 파견할 것으로 FINA로부터 만리적 초대를 받았다.

Korean -> English:
'The competition consists of two rounds: preliminaries and finals. The top 8 relay teams with the best times in the preliminaries advance to the finals. If necessary, swimming matches are used to break ties and advance to the next round.'


#### Tarea para Estudiantes:
Transformación de Tono

La escritura puede variar según la audiencia objetivo. ChatGPT puede producir diferentes tonos. Transforma el siguiente mensaje en una carta de negocios.

In [62]:
original_message = "David, it's John! OMG, the Olympic game is so crazy"

In [63]:
# Escribe aquí el código para la tarea .
message = [{'role': 'system', 'content': '''
You're an assistant designed to transform text to the requested context.
Users will paste in a string of text and you'll transform text to the requested context.
Use the format: <original text>: \n <new context>:\n <transformed text>
'''},
{'role': 'user', 'content': f'Transform the following text to a bussines context: {original_message}'}]

response = get_completion_from_messages(message)

print(response)

Original text:
David, it's John! OMG, the Olympic game is so crazy

Business context:
Exciting news, David! The Olympic games are generating a lot of buzz and excitement.


#### Tarea para Estudiantes:
Conversión de Formato

ChatGPT puede traducir entre formatos. El prompt debe describir los formatos de entrada y salida. Convierte los siguientes datos JSON al formato HTML.

In [64]:
data_json = { "The 2020 Summer Olympics Opening Ceremony audience name list" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [65]:
# Escribe aquí el código para la tarea .
message = [{'role': 'system', 'content': '''
You are a assistant designed to transform formats. 
You have to identify the orignal format, describe it and transformed it to the requested format. 
'''},
{'role': 'user', 'content': f'Describe the format from {data_json} and transform it to html format.'
}]

response = get_completion_from_messages(message)
print(response)

The original format is a list of dictionaries containing information about audience members attending the 2020 Summer Olympics Opening Ceremony. Each dictionary includes the name and email address of an audience member.

Transformed to HTML format:
```html
<ul>
  <li>Name: Shyam, Email: shyamjaiswal@gmail.com</li>
  <li>Name: Bob, Email: bob32@gmail.com</li>
  <li>Name: Jai, Email: jai87@gmail.com</li>
</ul>
```


#### Tarea para Estudiantes:
Revisión ortográfica y gramatical del siguiente texto.

Para indicarle al modelo LLM que deseas que revise tu texto, instruye al modelo para que 'revise' o 'revise y corrija'.

In [66]:
check_text = [ 
  "Alongside the main Emblem blue, the five other colors use in the branding of the 2020 Games is : Kurenai red,   Ai blue,   Sakula pink,   Fuji purple, and   Matsuba green.",
  "The competition have three round: heats, semifinals, and a final."
]

In [67]:
# Escribe aquí el código para la tarea .
message = [{'role': 'system', 'content': '''
            You're an assistant designed to check the grammar of the given text.
            Users will paste in a string of text and you'll respond with a grammar check.
            '''},
            {'role': 'user', 'content': f'Check the grammar of the following text: {check_text}'}]  

response = get_completion_from_messages(message)
print(response)

Here are the corrected sentences:

1. Alongside the main Emblem blue, the five other colors used in the branding of the 2020 Games are: Kurenai red, Ai blue, Sakula pink, Fuji purple, and Matsuba green.
2. The competition has three rounds: heats, semifinals, and a final.


#### Táctica 5: Ampliación: personaliza la respuesta automática

#### Tarea para Estudiantes:
Personaliza la respuesta automática al siguiente correo electrónico de un cliente.

El cliente tuvo un problema al comprar un boleto para los Juegos Olímpicos.

In [68]:
# given the sentiment from the tactic on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a ticket transaction
review = f"""
I bought the ticket of "Men's 100 metre freestyle swimming" game last week.\
The transaction went through successfully. However, I still have not received the ticket.\
Over one week has passed.\
"""

In [69]:
# Escribe aquí el código para la tarea .
message = [{'role': 'system', 'content': '''
            You're an assistant designed to customize email based on sentiment.
            Users will paste in a string of text and you'll respond with a customized email.
            '''},
            {'role': 'user', 'content': f'Customize the email based on the sentiment {sentiment} and the original customer message: {review}'}]

response = get_completion_from_messages(message)
print(response)

Subject: Urgent: Issue with Ticket Purchase for "Men's 100 Metre Freestyle Swimming" Game

Dear [Customer's Name],

I hope this email finds you well. 

I am writing to address the issue you have encountered regarding the purchase of your ticket for the "Men's 100 Metre Freestyle Swimming" game. I am truly sorry to hear about the delay in receiving your ticket despite the successful transaction.

Rest assured, I have escalated this matter to our support team to investigate and resolve the issue promptly. Your satisfaction is our top priority, and we are committed to ensuring that you receive your ticket in time for the event.

Please accept our sincerest apologies for any inconvenience this has caused you. We appreciate your patience and understanding as we work to rectify this situation.

Should you have any further concerns or require assistance, please do not hesitate to contact me directly at [Your Contact Information].

Thank you for bringing this to our attention, and we look forw

#### Táctica 6: Chatbot: personalizar conversaciones para tareas o comportamientos específicos

#### Tarea para Estudiantes:
Crear una conversación con el chatbot para saber dónde se celebraron los Juegos Olímpicos de Verano de 2020.

In [70]:
# Escribe aquí el código para la tarea 
message = [{'role': 'system', 'content': '''
            You're an friendly assistant
            '''},
            {'role': 'user', 'content': f'Hi, I am passionate about the Olimpic Games.'},
            {'role': 'assistant', 'content': 'That is great, i can give you some information about the Olimpic Games. How can I help you today?'},
            {'role': 'user', 'content': f'Can you tell me about the 2020 Summer Olympics?'}]

response = get_completion_from_messages(message)
print(response)

The 2020 Summer Olympics, which were originally scheduled to take place in Tokyo, Japan, were postponed to 2021 due to the COVID-19 pandemic. The games are now set to take place from July 23 to August 8, 2021. This will be the second time that Tokyo has hosted the Summer Olympics, with the first time being in 1964. The Olympics will feature a wide range of sports and competitions, showcasing the talents of athletes from around the world. Is there anything specific you would like to know about the 2020 Summer Olympics?


### 2.2 Dale tiempo al modelo para "pensar"

#### Táctica 1: Especificar los pasos necesarios para completar una tarea
A veces puedes ayudar al modelo a "ralentizarse" y ofrecer respuestas más robustas y detalladas especificando los pasos que debe seguir.

Pidamos la salida en varios formatos especificados.

In [71]:
prompt = f"""
Your task is to help a journalist summarize information from the article for publication.

Write a title based on the information provided in the context delimited by triple backticks. 
The title should be short, catchy, and reflective of the article's narrative.

After the title, generate five keywords from the context.

After the keywords, include a table to organize the information. 
The table should have two columns. In the first column is the title.
In the second column include the keywords as a list.

Give the table the title 'Article Publishing Information'.

Format everything as HTML that can be used in a website.
Place the title in a <div> element.

Context: ```{text}

""" 

get_chat_completion(prompt)

'<div>Title: Tokyo 2020 Olympics Highlights</div>\n\nKeywords:\n1. Tokyo 2020\n2. Olympic Games\n3. COVID-19 pandemic\n4. New sports\n5. Medal count\n\n<table>\n  <tr>\n    <th>Article Publishing Information</th>\n    <th>Keywords</th>\n  </tr>\n  <tr>\n    <td>Tokyo 2020 Olympics Highlights</td>\n    <td>Tokyo 2020, Olympic Games, COVID-19 pandemic, New sports, Medal count</td>\n  </tr>\n</table>'

#### Táctica 2: Instruir al modelo para que desarrolle su propia solución

Existen dos métodos principales que demostraremos en esta sección para lograr que el modelo resuelva un problema en lugar de apresurarse a una conclusión: cadena de pensamiento (chain-of-thought) y encadenamiento (chaining). Estas estrategias pueden llevar a una mayor precisión, detalle y la capacidad de resolver desafíos complejos.


2.1 - Prompting de cadena de pensamiento (Chain-of-thought)
- Pedir al modelo que razone
- Ejemplo de un solo disparo (one-shot)
- Razonamiento del chatbot

2.2 - Encadenamiento (Chaining)

Continuemos trabajando con el conjunto de datos de los Juegos Olímpicos.

#### 2.2.1 Prompting de Cadena de Pensamiento (Chain-of-Thought Prompting)

Hagamos un poco de matemáticas. Los modelos de GPT ocasionalmente no resuelven muy bien problemas matemáticos directos, así que guiemos al modelo a través del problema.

Descomponemos las tareas en piezas más pequeñas.

Lee más sobre los métodos y la investigación en el documento técnico: https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md

#### Táctica 1: Puedes comenzar pidiéndole específicamente al modelo que piense paso a paso.

In [72]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

# From Azure documentation
prompt = "Who was the most decorated (maximum medals) individual athlete in the Olympic games that were held at Sydney? Take a step-by-step approach in your response, cite sources and give reasoning before sharing final answer in the below format: ANSWER is: <name>"
get_chat_completion(prompt)

'Step 1: Identify the athlete who won the most medals at the Sydney Olympics\n\nStep 2: Look up the medal count for the top-performing athletes at the Sydney Olympics\n\nStep 3: Compare the medal counts to determine the most decorated athlete\n\nStep 4: Provide the final answer with reasoning and sources\n\nAfter conducting research, the most decorated individual athlete at the Sydney Olympics was Ian Thorpe from Australia.\n\nIan Thorpe won a total of 5 medals at the Sydney Olympics in 2000. He won 3 gold medals (in the 400m freestyle, 4x100m freestyle relay, and 4x200m freestyle relay), 1 silver medal (in the 200m freestyle), and 1 bronze medal (in the 100m freestyle).\n\nTherefore, the ANSWER is: Ian Thorpe'

In [73]:
# Another example
prompt = f"""
What is the largest time zone difference is between the top two countries who 
won the most gold medals in the 2020 Tokyo olympics?

Use the context below and think aloud as you solve the problem, step-by-step.

Context: {text}
"""
get_chat_completion(prompt)

'To find the largest time zone difference between the top two countries who won the most gold medals in the 2020 Tokyo Olympics, we need to identify the top two countries and then determine the time zone difference between them.\n\nAccording to the context provided, the United States topped the medal count with 39 gold medals, and China finished second with 38 gold medals. Therefore, the top two countries are the United States and China.\n\nThe United States is in multiple time zones, but for the purpose of this calculation, we can use the Eastern Time Zone (UTC-4) as a reference. China is in a single time zone, which is China Standard Time (UTC+8).\n\nTo find the time zone difference between the United States (Eastern Time Zone) and China, we can subtract the UTC offset of the Eastern Time Zone from the UTC offset of China Standard Time:\nUTC+8 (China) - UTC-4 (Eastern Time) = 12 hours\n\nTherefore, the largest time zone'

#### Táctica 2: Ejemplo de un solo disparo (One-shot)
Otra táctica común es proporcionar un ejemplo de una consulta y una respuesta ideal. El modelo aprenderá de ese ejemplo y aplicará los patrones a una nueva pregunta.

In [74]:
# Notice how this response may not be ideal, or the most accurate.
prompt = f"""
    The United States has 113 total medals, 39 of which are gold medals. 
    
    Great Britain has 64 medals and 22 gold medals. 
    
    How many more silver and bronze medals does the United States have over Great Britain?
"""

get_chat_completion(prompt)

'The United States has 113 total medals - 39 gold medals = 74 silver and bronze medals.\nGreat Britain has 64 total medals - 22 gold medals = 42 silver and bronze medals.\n\nTherefore, the United States has 74 - 42 = 32 more silver and bronze medals than Great Britain.'

In [75]:
# Give the model a one-shot example to solve the question more thoroughly

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the following question using similar steps above.

Question: China has 89 total medals, 38 of which are gold medals. How many silver and bronze medals do they have?
Answer:
"""

get_chat_completion(prompt)

'[Step 1] - There are three types of medals: gold, silver, and bronze\n[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (38) and total (89) counts. \n            89-38=51, so there are 51 silver and bronze medals combined. The answer is 51.'

#### Táctica 3: Razonamiento de cadena de pensamiento del chatbot (chain-of-thought reasoning)
Puedes incorporar lógica utilizando variables para que el chatbot pueda demostrar de manera dinámica formas específicas de pensar sobre un problema.

**Para ingresar la información, aparecerá un cuadro de texto en la parte superior de la pantalla. Deberías ver una ventana emergente. Escribe 'quit' si quieres salir.**

In [76]:
# Ask the bot to help you make a decision such as deciding whether to take a job or choose between restaurants.
# If the model does not immediately respond to your query, wait 1-5 seconds and retype it.
# If it is not allowing you to give an input, restart the kernel in the navigation bar.
# Type "quit" to end the session

context = '''
        You are a decision bot. Your job is to help come to decision by asking a series of questions one at a time and coming to a reasonable decision based on the information provided.

        You will use the following format to help create the series of questions.

        Template: 
        [Problem/Scenario/Question]: [Provide a brief description of the problem, scenario, or question.]

        Chain of thought:

        [Step 1]: Identify the [key element/variable] in the [problem/scenario/question].
        [Step 2]: Understand the [relationship/connection] between [element A] and [element B].
        [Step 3]: [Analyze/Evaluate/Consider] the [context/implication] of the [relationship/connection] between [element A] and [element B].
        [Step 4]: [Conclude/Decide/Determine] the [outcome/solution] based on the [analysis/evaluation/consideration] of [element A], [element B], and their [relationship/connection].
        [Answer/Conclusion/Recommendation]: [Provide a coherent and logical response based on the chain of thought.]

        You will guide the user though a series of questions one at a time. The first question is broad, and they subsequent questions become more specific. 

        Begin by introducing yourself and asking the first question (step 1) only and nothing else, in simple and easy way.
        '''

conversation=[{"role": "system", "content": context}]

while(True):
    print(len(conversation))
    if len(conversation) == 1:
        response = get_completion_from_messages(conversation)
        conversation.append({"role": "assistant", "content": response})
        print("\n" + response + "\n")
    
    user_input = input('Enter your response: ')
    if user_input.lower() == "quit":
        break 
    conversation.append({"role": "user", "content": user_input})
    
    response = get_completion_from_messages(conversation)
    conversation.append({"role": "assistant", "content": response})
    print("\n" + response + "\n")


1

Hello! How can I assist you today?

[Problem/Scenario/Question]: Are you trying to decide between two options?



#### 2.2.2 - Encadenamiento / Chaining
Similar a algunos ejemplos anteriores, puedes usar las salidas del modelo de consultas anteriores en otras consultas. Más adelante en el Hack te mostraremos cómo hacer esto a escala.

In [77]:
# Extract medal counts for each country from the news article
# Write an article about the summarized information
# Provide a title for the summary

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

prompt = f"""
    Write a brief article about the winners and losers of the Olympics based on medal count:
    {num_medals_dict}
"""
summary = get_chat_completion(prompt)
print(summary)

prompt = f"""
    Give the summary a title in 5 words:
    {summary}
"""
title = get_chat_completion(prompt)
print(title)

The 2020 Tokyo Olympics have come to a close, and it's time to reflect on the winners and losers based on the medal count. The United States emerged as the clear winner, dominating the competition with a total of 113 medals, including an impressive 39 gold medals. This performance solidified their position as the top medal-winning country at the Games.

China also had a strong showing, finishing in second place with a total of 89 medals, including 38 gold medals. Japan, the host country, performed admirably with 27 gold medals and a total of 58 medals overall. Great Britain and the Russian Olympic Committee (ROC) rounded out the top five medal-winning countries, with 64 and 71 total medals respectively.

On the other end of the spectrum, Bermuda and the Philippines each won one gold medal, making them the smallest medal-winning countries at the Games. While these countries may not have topped the medal count, their athletes' achievements are still worth celebrating.

Overall
United Sta

#### Tarea para Estudiantes (Encadenamiento / Chaining):

Tu tarea ahora es escribir código que determine el país que ganó la mayor cantidad de medallas de plata y bronce combinadas.

Podemos ver que el modelo tiene un rendimiento deficiente al responder la pregunta directamente.

In [84]:
prompt = f"""
    Based on the context, which country had the most silver and bronze medals?
    Context: {text}
"""

get_chat_completion(prompt)

'Based on the context, the United States had the most silver and bronze medals at the 2020 Summer Olympics, with a total of 39 gold medals, 41 silver medals, and 33 bronze medals.'

Escribe prompts en la celda a continuación que ayudarán al modelo a responder la pregunta desglosando las tareas en diferentes pasos. Asegúrate de que responda solo con la información que se le proporcionó. Este concepto de fundamentación se presentará con más detalle en el Reto 3.

Deberías poder lograr que el modelo responda la pregunta en 2-3 pasos.

In [83]:
# Escribe aquí el código para la tarea .
message = [
    {
        'role': 'system',
        'content': '''
            You are an assistant with expertise in analyzing Olympic data. 
            - Identify and extract key data from the provided context as requested.
            - Only use the context provided. 
            - Explain the reasoning step by step.
            Then answer
            - Format the final response as: Answer: <answer>
        '''
    },
    {
        'role': 'user',
        'content': f'''
            Based on the context, which country had the most silver and bronze medals? 
            Context: {text}
        '''
    }
]

# Llamada a la función que interactúa con la API de OpenAI
response = get_completion_from_messages(message)

# Imprime la respuesta generada
print(response)

To determine which country had the most silver and bronze medals based on the provided context, we need to look for information on the total number of silver and bronze medals won by each country at the 2020 Summer Olympics.

1. The context mentions that the United States topped the medal count by both total golds and total medals, with 39 gold medals and 113 total medals.
2. It further states that China finished second with 38 gold medals and 89 total medals.
3. Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 gold medals and 58 total medals.
4. Great Britain finished fourth with a total of 22 gold medals and 64 medals.

Based on this information, we can calculate the total number of silver and bronze medals for each country:
- United States: 113 total medals - 39 gold medals = 74 silver and bronze medals
- China: 89 total medals - 38 gold medals = 51 silver and bronze medals
- Japan: 58 total med

## Conclusión

En este primer desafío, cubrimos cómo interactuar con Azure OpenAI para diferentes objetivos. Esperamos que hayas podido ver la versatilidad dinámica de los modelos y cómo pueden usarse para resolver una variedad de problemas utilizando diferentes técnicas.

Le dimos a la API piezas cortas de texto usando variables fijas. En el próximo conjunto de desafíos, verás cómo usar la API con conjuntos de datos más grandes.